In [1]:
import numpy as np
from resonance.nonlinear_systems import SingleDoFNonLinearSystem

To apply arbitrary forcing to a single degree of freedom linear or nonlinear system, you can do so with `SingleDoFNonLinearSystem` (`SingleDoFLinearSystem` does not support arbitrary forcing...yet).

Add constants, a generalized coordinate, and a generalized speed to the system.

In [2]:
sys = SingleDoFNonLinearSystem()

In [3]:
sys.constants['m'] = 100 # kg
sys.constants['c'] = 1.1*1.2*0.5/2
sys.constants['k'] = 10
sys.constants['Fo'] = 1000 # N
sys.constants['Ft'] = 100 # N/s
sys.constants['to'] = 3.0  # s

sys.coordinates['x'] = 0.0
sys.speeds['v'] = 0.0

Create a function that evaluates the first order form of the non-linear equations of motion. In this case:

$$
\dot{x} = v \\
m\dot{v} + c \textrm{sgn}(v)v^2 + k \textrm{sgn}(x)x^2 = F(t)
$$

Make the arbitrary forcing term, $F$, an input to this function.

In [4]:
def eval_eom(x, v, m, c, k, F):
    xdot = v
    vdot = (F - np.sign(v)*c*v**2 - np.sign(x)*k*x**2) / m
    return xdot, vdot

Note that you cannot add this to the system because `F` has not been defined.

In [5]:
sys.diff_eq_func = eval_eom

KeyError: 'F is not in constants, coordinates, speeds, or measurements.'

To rememdy this, create a function that returns the input value given the appropriate constants and time.

In [6]:
def eval_step_input(Fo, to, time):
    if time < to:
        return 0.0
    else:
        return Fo

In [7]:
import matplotlib.pyplot as plt
%matplotlib widget

In [8]:
ts = np.linspace(0, 10)
plt.plot(ts, eval_step_input(5.0, 3.0, ts))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [9]:
ts < 3.0

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [10]:
def eval_step_input(Fo, to, time):
    F = np.empty_like(time)
    for i, ti in enumerate(time):
        if ti < to:
            F[i] = 0.0
        else:
            F[i] = Fo
    return F

In [11]:
plt.plot(ts, eval_step_input(5.0, 3.0, ts))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
eval_step_input(5.0, 3.0, ts)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [13]:
eval_step_input(5.0, 3.0, 7.0)

TypeError: 'float' object is not iterable

In [14]:
def eval_step_input(Fo, to, time):
    if np.isscalar(time):
        if time < to:
            return 0.0
        else:
            return Fo
    else:
        F = np.empty_like(time)
        for i, ti in enumerate(time):
            if ti < to:
                F[i] = 0.0
            else:
                F[i] = Fo
        return F

In [15]:
eval_step_input(5.0, 3.0, 7.0)

5.0

In [16]:
eval_step_input(5.0, 3.0, ts)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [17]:
True * 5.0

5.0

In [18]:
False * 5.0

0.0

In [19]:
(ts >= 3.0)*5.0

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [20]:
(5.0 >= 3.0)*5.0

5.0

In [21]:
def eval_step_input(Fo, to, time):
    return (time >=to)*Fo

In [22]:
eval_step_input(5.0, 3.0, ts)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [23]:
eval_step_input(5.0, 3.0, 7.0)

5.0

In [24]:
sys.add_measurement('F', eval_step_input)

In [25]:
sys.diff_eq_func = eval_eom

In [26]:
traj = sys.free_response(20.0)

In [27]:
traj.plot(subplots=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f59e44733c8>,
      dtype=object)

In [28]:
def eval_ramp_input(Ft, to, time):
    return (time >= to)*(Ft*time - Ft*to)

In [29]:
del sys.measurements['F']

In [30]:
sys.add_measurement('F', eval_ramp_input)

In [31]:
sys.measurements

{'F': -0.0}

In [32]:
traj = sys.free_response(20.0)

In [33]:
traj.plot(subplots=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f59e235b2e8>,
      dtype=object)